In [1]:
#|default_exp img_sen2

# Sentinel 2

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils

sentinel-2 class

In [4]:
#|export

class access_s2_image():
  
  def __init__(self,
               path_to_geometry : str , # path to geometry file, can be .shp or .gpkg . example : r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg"
               crs : str , #coordinate reference system to work with
               ):
    
    self.crs = crs
    self.gdf = utils.read_gdf(geometry_path=path_to_geometry,crs=self.crs)

    

In [6]:
extractor = access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", crs = "4326")

extractor.gdf

,id,name,geometry
0,1,1,"MULTIPOLYGON (((34.19874 31.59613, 34.63046 31..."
